In [1]:
import cv2
import numpy as np
from keras.datasets import cifar10
from keras import backend as K
from keras.utils import np_utils

nb_train_samples = 50000 # 5000 training samples
nb_valid_samples = 10000 # 1000 validation samples
num_classes = 10

def load_cifar10_data(img_rows, img_cols):

    # Load cifar10 training and validation sets
    (X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()

    # Resize trainging images
    if K.image_dim_ordering() == 'th':
        X_train = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in X_train[:nb_train_samples,:,:,:]])
        X_valid = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in X_valid[:nb_valid_samples,:,:,:]])
    else:
        X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:nb_train_samples,:,:,:]])
        X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:nb_valid_samples,:,:,:]])

    # Transform targets to keras compatible format
    Y_train = np_utils.to_categorical(Y_train[:nb_train_samples], num_classes)
    Y_valid = np_utils.to_categorical(Y_valid[:nb_valid_samples], num_classes)

    return X_train, Y_train, X_valid, Y_valid

Using TensorFlow backend.


In [0]:
X_train, Y_train, X_valid, Y_valid = load_cifar10_data(32,32)

In [3]:
print(X_train.shape, X_valid.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3)


In [0]:
from keras.applications import vgg16
from keras import Sequential
from keras import layers,optimizers

In [6]:
vgg16_notop = vgg16.VGG16(weights='imagenet',include_top=False,input_shape=(32,32,3))
vgg16_notop.summary()

58892288/58889256 [==============================] - 2s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool

In [9]:
model = Sequential()
for layer in vgg16_notop.layers:
  layer.trainable=False
  model.add(layer)
#model.add(vgg16_conv)
model.add(layers.Flatten())
model.add(layers.Dense(1024,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10,activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 8, 8, 256)         295168    
__________

In [0]:
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [0]:
model.compile(optimizer=Adam(lr=0.00001),loss='categorical_crossentropy',metrics=['accuracy'])

In [20]:
model.fit(X_train,Y_train,batch_size=200,epochs=10,validation_split=0.1,verbose=2)

Train on 45000 samples, validate on 5000 samples
Epoch 1/10
 - 14s - loss: 1.3945 - acc: 0.5064 - val_loss: 1.2935 - val_acc: 0.5932
Epoch 2/10
 - 14s - loss: 1.3934 - acc: 0.5088 - val_loss: 1.2969 - val_acc: 0.5872
Epoch 3/10
 - 14s - loss: 1.3903 - acc: 0.5097 - val_loss: 1.2908 - val_acc: 0.5920
Epoch 4/10
 - 14s - loss: 1.3855 - acc: 0.5120 - val_loss: 1.2958 - val_acc: 0.5900
Epoch 5/10
 - 14s - loss: 1.3797 - acc: 0.5118 - val_loss: 1.3003 - val_acc: 0.5976
Epoch 6/10
 - 14s - loss: 1.3770 - acc: 0.5143 - val_loss: 1.2848 - val_acc: 0.5948
Epoch 7/10
 - 14s - loss: 1.3708 - acc: 0.5160 - val_loss: 1.2795 - val_acc: 0.5886
Epoch 8/10
 - 14s - loss: 1.3682 - acc: 0.5140 - val_loss: 1.2944 - val_acc: 0.5870
Epoch 9/10
 - 14s - loss: 1.3511 - acc: 0.5195 - val_loss: 1.2878 - val_acc: 0.5862
Epoch 10/10
 - 14s - loss: 1.3535 - acc: 0.5198 - val_loss: 1.2809 - val_acc: 0.5950
